# [STARTER] Exercise - Output structured Agent responses

In this exercise, you'll learn how to enhance your AI agent to provide structured outputs using Pydantic models. This will help ensure the agent's responses are consistent, validated, and easily usable in downstream applications.

## Challenge

You have an existing Agent class that can:
- Process user messages
- Use tools when needed
- Generate responses

Now you need to enhance it to:
- Define structured output formats using Pydantic
- Parse and validate responses
- Return data in a consistent JSON format



## Setup
First, let's import the necessary libraries:

In [1]:
from typing import List, Any, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

from lib.messages import UserMessage, SystemMessage, ToolMessage
from lib.tooling import tool
from lib.llm import LLM
from lib.parsers import PydanticOutputParser, JsonOutputParser

## Defining Structured Output Models

Let's create a Pydantic model for a meeting summary with action items:


In [2]:
# TODO 1: Create the ActionItem Pydantic model
# Hint: Include fields for task, assignee, and due_date with appropriate annotations and descriptions

class ActionItem(BaseModel):
    """A Pydantic model representing an action item.
    
    This model defines the structure for action items with validation:
    - task: The task to be completed
    - assignee: The person assigned to complete the task
    - due_date: The date the task is due
    """

    task: Annotated[str, Field(description="The task to be completed. Defaults to ''", default=None)]
    assignee: Annotated[str, Field(description="The person assigned to complete the task. Defaults to ''", default=None)]
    due_date: Annotated[str, Field(description="The date the task is due. Defaults to ''", default=None)]

In [3]:
# TODO 2: Create the MeetingSummary Pydantic model
# Hint: Include fields for title, date, participants, key_points, and action_items

class MeetingSummary(BaseModel):
    """A Pydantic model representing a meeting summary.
    
    This model defines the structure for meeting summaries with validation:
    - title: The title of the meeting
    - date: The date of the meeting
    - participants: List of participants in the meeting
    - key_points: List of key points discussed
    - action_items: List of action items to be completed
    """

    title: Annotated[str, Field(description="The title of the meeting. Defaults to ''", default=None)]
    date: Annotated[str, Field(description="The date of the meeting. Defaults to ''", default=None)]
    participants: Annotated[List[str], Field(description="List of participants in the meeting. Defaults to []", default=None)]
    key_points: Annotated[List[str], Field(description="List of key points discussed. Defaults to []", default=None)]
    action_items: Annotated[List[ActionItem], Field(description="List of action items to be completed. Defaults to []", default=None)]


## Enhanced Agent Class

Now let's create an enhanced version of our Agent class that supports structured outputs:


In [4]:
class StructuredAgent:
    """An AI Agent that provides structured outputs"""
    
    def __init__(
        self,
        role: str = "Meeting Assistant",
        instructions: str = "Help summarize meetings and track action items",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[Any] = None,
        output_model: BaseModel = None
    ):
        """Initialize the agent with its configuration
        
        Args:
            role: The agent's role/persona
            instructions: Basic instructions for the agent
            model: The LLM model to use
            temperature: Creativity parameter (0.0 = more deterministic)
            tools: List of tools the agent can use
            output_model: Pydantic model for structured output
        """
        # TODO 3: Initialize the agent
        # Hint:
        # - Store agent settings (role, instructions, output_model, etc.)
        # - Load environment variables
        # - Create an LLM instance with the provided configuration
        self.role = role
        self.instructions = instructions
        self.model = model                
        self.output_model = output_model
        self.llm = LLM(model=model, temperature=temperature, tools=tools) 

        load_dotenv(override=True)


    def invoke(self, user_message: str) -> dict:
        """Process a user message and return a structured response
        
        Args:
            user_message: The user's input message
            
        Returns:
            A dictionary containing the structured response
        """
        # TODO 4: Implement the invoke method
        # Hint:
        # - Create messages list with SystemMessage
        # - Add UserMessage
        # - Get AI response with structured format if output_model exists
        # - Parse and return the response
        messages = [
            SystemMessage(content=f"You're an AI Agent and your role is {self.role}. Your instructions: {self.instructions}")
        ]
        messages.append(UserMessage(content=user_message))

        if self.output_model:
            ai_message = self.llm.invoke(messages, response_format=self.output_model)
            parser = JsonOutputParser()
            return parser.parse(ai_message)

        
        ai_message = self.llm.invoke(messages)        

        return {
            "response": ai_message.content
        }
        
            


## Testing the Structured Agent

Let's test our enhanced agent with a meeting summary example:


In [5]:
# Create an agent instance with the MeetingSummary model
meeting_agent = StructuredAgent(
    role="Meeting Assistant",
    instructions="Summarize meetings and track action items in a structured format",
    output_model=MeetingSummary
)

In [6]:
meeting_transcript = """
Project Planning Meeting - March 15, 2024

Attendees: John, Sarah, Mike

Discussion:
- Reviewed Q1 project timeline
- Discussed resource allocation
- Identified potential risks

Next steps:
1. John will update the project plan by next Friday
2. Sarah needs to coordinate with the design team by Wednesday
3. Mike will prepare the risk assessment document by end of month
"""

In [7]:
summary = meeting_agent.invoke(meeting_transcript)
print(json.dumps(summary, indent=2))

{
  "title": "Project Planning Meeting",
  "date": "March 15, 2024",
  "participants": [
    "John",
    "Sarah",
    "Mike"
  ],
  "key_points": [
    "Reviewed Q1 project timeline",
    "Discussed resource allocation",
    "Identified potential risks"
  ],
  "action_items": [
    {
      "task": "Update the project plan",
      "assignee": "John",
      "due_date": "March 22, 2024"
    },
    {
      "task": "Coordinate with the design team",
      "assignee": "Sarah",
      "due_date": "March 20, 2024"
    },
    {
      "task": "Prepare the risk assessment document",
      "assignee": "Mike",
      "due_date": "March 31, 2024"
    }
  ]
}


## Validating the Output

Let's verify that our output matches our Pydantic model structure:


In [8]:
# Create a MeetingSummary instance from the output
validated_summary = MeetingSummary(**summary)

In [9]:
# Access structured data
print("Meeting Title:", validated_summary.title)
print("\nParticipants:")
for participant in validated_summary.participants:
    print(f"- {participant}")

print("\nAction Items:")
for item in validated_summary.action_items:
    print(f"- {item.task} (Assigned to: {item.assignee}, Due: {item.due_date})")

Meeting Title: Project Planning Meeting

Participants:
- John
- Sarah
- Mike

Action Items:
- Update the project plan (Assigned to: John, Due: March 22, 2024)
- Coordinate with the design team (Assigned to: Sarah, Due: March 20, 2024)
- Prepare the risk assessment document (Assigned to: Mike, Due: March 31, 2024)
